# 1. 서울 열린데이터 광장

[서울 열린데이터 광장](https://data.seoul.go.kr/)(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

# 2. 서울시 공공자전거 실시간 대여정보

1. 인증키를 발급 받습니다.

2. "서울시 공공자전거 실시간 대여정보" 를 검색합니다.

6a7458436967757336315578595679

In [1]:
import requests
import pandas as pd
import folium
import json
import warnings
warnings.filterwarnings('ignore')

In [9]:
urls = []
start = 1
end = 1000
for i in range(1, 4):
  url = f"http://openapi.seoul.go.kr:8088/6a7458436967757336315578595679/json/bikeList/{start}/{end}/"
  start = (1000 * i) + 1
  end = (1000*i) + 1000
  urls.append(url)

In [18]:
datas = []
for url in urls:
  req = requests.get(url)
  data = req.json()
  json_data = data.get("rentBikeStatus", {}).get("row", "")
  datas.append(json_data)

In [24]:
x, y = datas[0][0]["stationLatitude"], datas[0][0]["stationLongitude"]

In [28]:
def fetch_bike():
  base_url = "http://openapi.seoul.go.kr:8088/6a7458436967757336315578595679/json/bikeList/"
  start = 1
  end = 1000
  step = 1000
  data_frames = []

  while True:
      url = f"{base_url}{start}/{end}/"
      response = requests.get(url)

      if response.status_code != 200:
          print(f"Status Code: {response.status_code}")
          break

      json_data = response.json()

      try:
          rent_bike_status = json_data["rentBikeStatus"]
          result_code = rent_bike_status["RESULT"]["CODE"]
      except KeyError:
          print("JSON 오류")
          break

      if result_code == "INFO-200":
          print("데이터 없음")
          break
      elif result_code == "INFO-000":
          print(f"시작: {start} 끝: {end}.")
          try:
              bike_data = rent_bike_status["row"]
              if bike_data:
                  df = pd.DataFrame(bike_data)
                  data_frames.append(df)
          except KeyError:
              print("데이터를 찾을 수 없음")
      else:
          print(f"result code: {result_code}")
          break

      start += step
      end += step

  if data_frames:
      final_df = pd.concat(data_frames, ignore_index=True)
      return final_df
  else:
      return pd.DataFrame()


In [29]:
bike_data = fetch_bike()

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


In [31]:
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758 entries, 0 to 2757
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2758 non-null   object
 1   stationName        2758 non-null   object
 2   parkingBikeTotCnt  2758 non-null   object
 3   shared             2758 non-null   object
 4   stationLatitude    2758 non-null   object
 5   stationLongitude   2758 non-null   object
 6   stationId          2758 non-null   object
dtypes: object(7)
memory usage: 151.0+ KB


```
rackTotCnt  거치대개수
parkingBikeTotCnt   자전거주차총건수
shared  거치율
stationLatitude 위도
stationLongitude    경도
stationId   대여소ID
stationName 대여소이름
```

In [34]:
bike_data[["stationLatitude", "stationLongitude"]] = bike_data[["stationLatitude", "stationLongitude"]].astype(float)
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758 entries, 0 to 2757
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rackTotCnt         2758 non-null   object 
 1   stationName        2758 non-null   object 
 2   parkingBikeTotCnt  2758 non-null   object 
 3   shared             2758 non-null   object 
 4   stationLatitude    2758 non-null   float64
 5   stationLongitude   2758 non-null   float64
 6   stationId          2758 non-null   object 
dtypes: float64(2), object(5)
memory usage: 151.0+ KB


In [38]:
bike_map = folium.Map(location=[bike_data.stationLatitude.mean(), bike_data.stationLongitude.mean()], zoom_start=14)

for index, data in bike_data.iterrows():
  popup_str = f"{data.stationName} 자전거 주차 총 건수 : {data.parkingBikeTotCnt}"
  popup = folium.Popup(popup_str, max_width=600)
  folium.Marker(location=[data.stationLatitude, data.stationLongitude], popup=popup).add_to(bike_map)

bike_map